In [20]:
import mysql.connector


def get_db_connection():
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="admin"
    )
    if mydb.is_connected():
        print("Connected to MySQL Server")
        cursor = mydb.cursor()
        cursor.execute("use Save;")
        return mydb, cursor
    else:
        return None, None

mydb,cursor = get_db_connection()
print(mydb)

Connected to MySQL Server


In [21]:
print(mydb.is_connected())

True


In [22]:
import pandas as pd

def process_accounts_file(mydb):
    if (mydb):
        # Providing a customer header, csv has inconsistent spaces in it's header row.
        df = pd.read_csv('account_info.csv', header=0, names=['first_name', 'last_name',
        'street_address', 'unit', 'city', 'state', 'zip', 'dob', 'ssn'
        ,'email_address','mobile_number','account_number'])

        #print(df.dtypes)

        for index, row in df.iterrows():
            # remove # from unit and put NULL for NAN records(DB requirement).
            unit = str(row['unit']).replace("#", "")
            if (str(row['unit']).upper()== 'NAN'):
                unit = "NULL"


            #Convert DOB to MySQL datetime format
            dob_split = row['dob'].split("/")
            if len(dob_split[0]) == 1:
                dob_split[0] = "0"+dob_split[0]
            if len(dob_split[1]) == 1:
                dob_split[1] = "0"+dob_split[1]
            dob = dob_split[2]+"-"+dob_split[0]+"-"+dob_split[1]

            query_string = "insert into accounts (first_name, last_name, street_address, unit, city, state, zip, dob, ssn, email_address, mobile_number, account_number) values (\"{0}\", \"{1}\", \"{2}\", {3}, \"{4}\", \"{5}\", \"{6}\", \"{7}\", \"{8}\", \"{9}\", {10}, {11});".format(row['first_name'], row['last_name'], row['street_address'], unit, row['city'], row['state'], row['zip'], dob, row['ssn'], row['email_address'], row['mobile_number'], row['account_number'])

            #print(query_string)
            cursor.execute(query_string)

        # Commit changes to db
        mydb.commit()
    else:
        print("DB connection unsuccessful.")
process_accounts_file(mydb)            
 

In [23]:

def process_transactions_file(mydb):
    if (mydb):
        # Providing a customer header, csv has inconsistent spaces in it's header row.
        df = pd.read_csv('transactions.csv', header=0, names=['account_number',
        'transaction_datetime', 'transaction_amount', 'post_date', 'merchant_number',
        'merchant_description','merchant_category_code','transaction_number'])

        #print(df.dtypes)

        for index, row in df.iterrows():

            #Put transaction_datetime to DB datetime format
            t_dt_split = row['transaction_datetime'].split(" ")
            # year-month-day
            t_date = t_dt_split[0][-4:] + "-" + t_dt_split[0][-8:-6] + "-" + t_dt_split[0][-6:-4] 

            transaction_datetime = t_date + " " + t_dt_split[1]

            # clean transaction amount
            transaction_amount = float(row['transaction_amount'][-1]+row['transaction_amount'][0:-1])

            #format post_date to DB date format
            post_date = str(row['post_date'])[-4:] + "-" + str(row['post_date'])[-8:-6] + "-" + str(row['post_date'])[-6:-4]

            """
            insert into transactions (account_number, transaction_datetime, transaction_amount, post_date, merchant_number, merchant_description, merchant_category_code, transaction_number) 
    values (1222, "2010-12-12 21:10:10", 100.100, "2010-10-10", 10000, "DEF", 1000, 1000);

            """

            query_string = "insert into transactions (account_number, transaction_datetime, transaction_amount, post_date, merchant_number, merchant_description, merchant_category_code, transaction_number) values ({0}, \"{1}\", {2}, \"{3}\", \"{4}\", \"{5}\", {6}, {7});".format(row['account_number'], transaction_datetime, transaction_amount, post_date, row['merchant_number'], row['merchant_description'], row['transaction_number'], row['merchant_category_code'])

            #print(query_string)
            cursor.execute(query_string)
        # Commit changes to DB
        mydb.commit()
    else:
        print("DB connection unsuccessful.")
process_transactions_file(mydb)            
 

In [13]:
mydb.close()